In [89]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [90]:
driver_path = "/Users/diegoandre/drivers/chromedriver"  # Replace YOUR_USERNAME with your actual username

options = webdriver.ChromeOptions()
options.set_capability(
    'goog:loggingPrefs', {'performance': 'ALL', 'browser': 'ALL'}
)

driver = webdriver.Chrome(service=ChromeService(driver_path), options=options)


In [91]:
driver.set_page_load_timeout(10)

try:
    driver.get('https://www.sofascore.com/football/match/athletic-club-liverpool/UsAgb#id:14050560,tab:statistics')
except:
    pass

# loads all of the APIs in the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [92]:
logs_raw = driver.get_log('performance') # gets all of the APIs that are being called

In [93]:
logs = [json.loads(lr['message']) for lr in logs_raw]

In [94]:
# looping through every network response
shotmap_request_id = None
for log in logs:
    if (log['message']['method'] == 'Network.requestWillBeSentExtraInfo' and
        'shotmap' in log['message']['params'].get('headers', {}).get(':path', '')):
        shotmap_request_id = log['message']['params']['requestId']
        print(f"Found shotmap requestWillBeSentExtraInfo with ID: {shotmap_request_id}")
        print(f"Path: {log['message']['params']['headers'][':path']}")
        break


Found shotmap requestWillBeSentExtraInfo with ID: 19376.1810
Path: /api/v1/event/14050560/shotmap


In [95]:
# metadata about network response
log

{'message': {'method': 'Network.requestWillBeSentExtraInfo',
  'params': {'associatedCookies': [],
   'clientSecurityState': {'initiatorIPAddressSpace': 'Public',
    'initiatorIsSecureContext': True,
    'privateNetworkRequestPolicy': 'PreflightWarn'},
   'connectTiming': {'requestTime': 89423.763682},
   'headers': {':authority': 'www.sofascore.com',
    ':method': 'GET',
    ':path': '/api/v1/event/14050560/shotmap',
    ':scheme': 'https',
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'en-US,en;q=0.9',
    'baggage': 'sentry-environment=production,sentry-release=JkGA0UcvCw7nIAISgTNn1,sentry-public_key=d693747a6bb242d9bb9cf7069fb57988,sentry-trace_id=2bca2ccce9a97caefbc0567310187a04',
    'cache-control': 'max-age=0',
    'priority': 'u=1, i',
    'referer': 'https://www.sofascore.com/football/match/athletic-club-liverpool/UsAgb',
    'sec-ch-ua': '"Not;A=Brand";v="99", "Google Chrome";v="139", "Chromium";v="139"',
    'sec-ch-ua-mobil

In [96]:
shotmap_request_id

'19376.1810'

In [97]:
# Now use that request ID to get the shotmap data
if shotmap_request_id:
    shotmap = json.loads(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': shotmap_request_id})['body'])['shotmap']


In [98]:
shotmap

[{'player': {'name': 'Adama Boiro',
   'firstName': 'Adama Boiro',
   'slug': 'adama-boiro',
   'shortName': 'A. Boiro',
   'position': 'D',
   'jerseyNumber': '32',
   'userCount': 501,
   'id': 1398511},
  'isHome': False,
  'shotType': 'save',
  'situation': 'assisted',
  'playerCoordinates': {'x': 16.9, 'y': 36.6, 'z': 0},
  'bodyPart': 'right-foot',
  'goalMouthLocation': 'low-left',
  'goalMouthCoordinates': {'x': 0, 'y': 53.7, 'z': 8.2},
  'blockCoordinates': {'x': 0.59999999999999, 'y': 49.6, 'z': 0},
  'id': 5365445,
  'time': 86,
  'timeSeconds': 5130,
  'draw': {'start': {'x': 36.6, 'y': 16.9},
   'block': {'x': 49.6, 'y': 0.59999999999999},
   'end': {'x': 46.3, 'y': 0},
   'goal': {'x': 46.3, 'y': 91.8}},
  'reversedPeriodTime': 5,
  'reversedPeriodTimeSeconds': 270,
  'periodTimeSeconds': 2430,
  'incidentType': 'shot'},
 {'player': {'name': 'Mikel Jauregizar',
   'firstName': '',
   'lastName': '',
   'slug': 'mikel-jauregizar',
   'shortName': 'M. Jauregizar',
   'posit